## Part 1: Getting Toronto Postal Code and Neighborhood Information

In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
# get html for beautiful soup to parse
url =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Retrieve the Toronto postal code data from html
pcodes = list()
for tag in soup.find_all('td'):
    pcodes.append(tag.string)
print(pcodes, len(pcodes))

['M1A\n', 'Not assigned\n', 'Not assigned\n', 'M2A\n', 'Not assigned\n', 'Not assigned\n', 'M3A\n', 'North York\n', 'Parkwoods\n', 'M4A\n', 'North York\n', 'Victoria Village\n', 'M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n', 'M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n', 'M7A\n', 'Downtown Toronto\n', "Queen's Park, Ontario Provincial Government\n", 'M8A\n', 'Not assigned\n', 'Not assigned\n', 'M9A\n', 'Etobicoke\n', 'Islington Avenue, Humber Valley Village\n', 'M1B\n', 'Scarborough\n', 'Malvern, Rouge\n', 'M2B\n', 'Not assigned\n', 'Not assigned\n', 'M3B\n', 'North York\n', 'Don Mills\n', 'M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n', 'M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n', 'M6B\n', 'North York\n', 'Glencairn\n', 'M7B\n', 'Not assigned\n', 'Not assigned\n', 'M8B\n', 'Not assigned\n', 'Not assigned\n', 'M9B\n', 'Etobicoke\n', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n', 'M1C\n', 'Scarboroug

In [5]:
# convert data from list to array to reshape and create dataframe,
import pandas as pd
import numpy as np

tpc_array = np.array(pcodes[:-33])
tpc_rearray = tpc_array.reshape(-1,3)

df_tpc = pd.DataFrame(tpc_rearray, columns=['Postal_Code', 'Borough', 'Neighbourhood'])
df_tpc.head()

,Postal_Code,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [6]:
df_tpc['Postal_Code'] = df_tpc['Postal_Code'].str.replace('\\n','')
df_tpc['Borough'] = df_tpc['Borough'].str.replace('\\n','')
df_tpc['Neighbourhood'] = df_tpc['Neighbourhood'].str.replace('\\n','')
df_tpc.head()

,Postal_Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df_tpc = df_tpc[df_tpc.Borough != 'Not assigned']
df_tpc = df_tpc.reset_index(drop=True)
df_tpc

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
df_tpc.shape

(103, 3)

## Part 2: Getting Lattitude and Longitude

In [9]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [10]:
#tried using geocoder with the following code with no success - even using example code in the docs it gave - "Request Denied"
#import geocoder
#latitude=list()
#longitude=list()
#for pc in df_tpc['Postal_Code']:
#    g=geocoder.google('{}, Toronto, Ontario'.format(pc))
#    lat_lng=g.latlng
#    latitude.append(lat_lng[0])
#    longitude.append(lat_lng[1])
#df_ll=pd.DataFrame(list(zip(lattitude,longitude)),columns=['Lattitude','Longitude'])
#df_ll.head()

In [11]:
df_ll=pd.read_csv('Geospatial_Coordinates.csv')
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_ll.rename(columns={'Postal Code':'Postal_Code'},inplace=True)
df_ll.head()

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_pcll = pd.merge(df_tpc,df_ll,on='Postal_Code')
df_pcll.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3: Exploring and Clustering Data
### Analysis of the neighbourhoods of Toronto

In [15]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [64]:
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [17]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto latitude and longitude: {}, {}.'.format(latitude, longitude))

Toronto latitude and longitude: 43.6534817, -79.3839347.


In [19]:
# create map of Toronto Neighourhoods
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df_pcll['Latitude'], df_pcll['Longitude'], df_pcll['Borough'], df_pcll['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [38]:
CLIENT_ID = 'PYRV4LPYOCPHX2OUBUPCOSNKD4DLATNPETUIGRDWGK3TQG23' 
CLIENT_SECRET = 'EKARS4OL2TPIMUII2ZADI1VSQ4FDGBR4WTDZUPP5WK4NOGWO' 
VERSION = '20180605' 

In [49]:
# create function to get venues for all neighbourhoods and create dataframe
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
#run the function above
toronto_venues = getNearbyVenues(names=df_pcll['Neighbourhood'],
                                   latitudes=df_pcll['Latitude'],
                                   longitudes=df_pcll['Longitude']
                                  )

In [51]:
#get shape and first 5 rows of the dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2153, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [52]:
# venues by neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4


In [53]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
toronto_onehot.shape

(2153, 268)

In [55]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [56]:
toronto_grouped.shape

(95, 268)

In [57]:
# function to sort by the top venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
# running the function above and creating a dataframe to view the most common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Coffee Shop,Athletics & Sports,Pub,Sandwich Place,Yoga Studio,Discount Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Supermarket,Mobile Phone Shop,Sushi Restaurant,Ice Cream Shop,Restaurant,Deli / Bodega,Fried Chicken Joint
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Thai Restaurant,Comfort Food Restaurant,Pizza Place,Juice Bar,Japanese Restaurant,Fast Food Restaurant


In [95]:
# clustering the neighbourhoods with k-means
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
k_means = KMeans(init = "k-means++", n_clusters = kclusters, n_init = 12)
k_means.fit(toronto_grouped_clustering)
k_means.labels_[0:10] 

array([4, 4, 4, 3, 4, 4, 3, 4, 3, 4])

In [96]:
# add clustering labels to dataframe
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', k_means.labels_)
torclust = df_pcll
torclust = torclust.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
torclust.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Portuguese Restaurant,Hockey Arena,Pizza Place,Coffee Shop,French Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Arts & Crafts Store,Furniture / Home Store,Women's Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,Coffee Shop,Diner,Yoga Studio,Café,Bar,Bank,Sushi Restaurant,Restaurant,Fried Chicken Joint,Beer Bar


In [98]:
torclust['Cluster Labels'].value_counts()

4.0    53
3.0    32
1.0    10
0.0     3
2.0     1
Name: Cluster Labels, dtype: int64

In [99]:
torclust.shape

(103, 16)

In [100]:
# there are 4 neighbourhoods that are NaN rather than having a cluster label
# remove the NaNs and convert the cluster lables to integers for mapping
torclust.dropna(subset=['Cluster Labels'],axis=0,inplace=True)
torclust.reset_index(drop=True,inplace=True)
torclust['Cluster Labels']=torclust['Cluster Labels'].astype(int)
torclust.shape

(99, 16)

In [102]:
# mapping the clusters
tor_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, ven, cluster in zip(torclust['Latitude'], torclust['Longitude'], torclust['Neighbourhood'], torclust['Cluster Labels']):
    label = folium.Popup(str(ven) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(tor_clusters)
       
tor_clusters

The map indicates that the downtown neighborhoods are very similar with more variance as you move away from downtown.

In [103]:
#cluster 1
torclust.loc[torclust['Cluster Labels'] == 0, torclust.columns[[1] + list(range(5, torclust.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,York,0,Trail,Field,Tennis Court,Hockey Arena,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
47,North York,0,Gym,Pizza Place,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
80,Central Toronto,0,Gym,Tennis Court,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Deli / Bodega


In [104]:
#cluster 2
torclust.loc[torclust['Cluster Labels'] == 1, torclust.columns[[1] + list(range(5, torclust.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
19,York,1,Park,Women's Store,Pool,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
33,East York,1,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
49,North York,1,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
61,York,1,Convenience Store,Park,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
63,North York,1,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
74,Etobicoke,1,Park,Sandwich Place,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
82,Scarborough,1,Park,Playground,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
88,Downtown Toronto,1,Park,Playground,Trail,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio
94,Etobicoke,1,Park,River,Pool,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


Parks are an obvious theme of Cluster 2

In [105]:
#cluster 3
torclust.loc[torclust['Cluster Labels'] == 2, torclust.columns[[1] + list(range(5, torclust.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,North York,2,Baseball Field,Food Service,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Falafel Restaurant


In [107]:
#cluster 4
torclust.loc[torclust['Cluster Labels'] == 3, torclust.columns[[1] + list(range(5, torclust.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
7,East York,3,Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection,Pet Store,Café,Breakfast Spot,Bank,Pharmacy
9,North York,3,Pizza Place,Bakery,Pub,Japanese Restaurant,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Dog Run
10,Scarborough,3,Home Service,Bar,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dessert Shop
12,East York,3,Pharmacy,Athletics & Sports,Curling Ice,Dance Studio,Skating Rink,Beer Store,Park,General Travel,Golf Course,Eastern European Restaurant
16,Scarborough,3,Electronics Store,Restaurant,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Bank,Mexican Restaurant,Distribution Center,Diner
23,Downtown Toronto,3,Grocery Store,Café,Park,Baby Store,Diner,Candy Store,Italian Restaurant,Coffee Shop,Nightclub,Restaurant
24,Scarborough,3,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Drugstore
25,North York,3,Golf Course,Fast Food Restaurant,Athletics & Sports,Mediterranean Restaurant,Pool,Dog Run,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant
29,West Toronto,3,Bakery,Pharmacy,Café,Bank,Supermarket,Bar,Middle Eastern Restaurant,Furniture / Home Store,Music Venue,Grocery Store


In [108]:
#cluster 5
torclust.loc[torclust['Cluster Labels'] == 4, torclust.columns[[1] + list(range(5, torclust.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Portuguese Restaurant,Hockey Arena,Pizza Place,Coffee Shop,French Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,4,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
3,North York,4,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Arts & Crafts Store,Furniture / Home Store,Women's Store,Vietnamese Restaurant
4,Downtown Toronto,4,Coffee Shop,Diner,Yoga Studio,Café,Bar,Bank,Sushi Restaurant,Restaurant,Fried Chicken Joint,Beer Bar
6,North York,4,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Sporting Goods Shop,Café,Bike Shop,Dim Sum Restaurant,Sandwich Place
8,Downtown Toronto,4,Coffee Shop,Clothing Store,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Café,Cosmetics Shop,Diner,Plaza,Pizza Place
11,North York,4,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Sporting Goods Shop,Café,Bike Shop,Dim Sum Restaurant,Sandwich Place
13,Downtown Toronto,4,Café,Coffee Shop,Clothing Store,Restaurant,American Restaurant,Cocktail Bar,Cosmetics Shop,Department Store,Creperie,Beer Bar
15,Etobicoke,4,Pet Store,Beer Store,Pharmacy,Pizza Place,Coffee Shop,Convenience Store,Café,Shopping Plaza,Liquor Store,Gastropub
17,East Toronto,4,Asian Restaurant,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner


The cluster including downtown has a lot of coffee shops, cafes, and bars.